In [12]:
import requests
import re
from bs4 import BeautifulSoup
import jieba
import nltk
import string
import pandas as pd
from nltk.tokenize import word_tokenize
import csv
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

url = 'https://www.bbc.com/zhongwen/articles/c4gl97d2rzjo/simp'
site = requests.get(url)
site_soup = BeautifulSoup(site.text, 'html.parser')
# print(site_soup.prettify())

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tempu\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode o

In [2]:
site_p = site_soup.find_all('p')[0:100]
text = ""
punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."

for p in site_p:
    text += str(p.get_text())


text.translate(str.maketrans('', '', string.punctuation))
text_re = re.sub(r"[%s]+" %punc, "", text)

tokens = jieba.lcut(text_re)
tokens_pd = pd.DataFrame(tokens)
tokens_pd.drop_duplicates(inplace=True)
print(tokens_pd.shape)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tempu\AppData\Local\Temp\jieba.cache
Loading model cost 1.575 seconds.
Prefix dict has been built successfully.


(747, 1)


In [45]:
HSK_nums = ['1', '2', '3', '4', '5', '6', '7_9']

HSK = pd.DataFrame()
for hsk in HSK_nums:
    hsk_df = pd.read_csv("../data/Chinese/HSK/HSK"+hsk+".tsv", sep="\t", header=None)
    hsk_df.drop(0, axis=1, inplace=True)
    hsk_df.rename(columns={1: "character", 2: "pinyin", 3: "definition"}, inplace=True)
    hsk_df.insert(loc=2,column="HSK",value=[hsk]*hsk_df.shape[0])
    hsk_df.insert(loc=4, column="embedding", value=list(model.encode(hsk_df.iloc[:, 3])))
    HSK = pd.concat([HSK, hsk_df])
HSK.head()

,character,pinyin,HSK,definition,embedding
0,爱,ài,1,"love, like, be fond of, be keen on, cherish, b...","[0.37996453, 0.29299688, 0.55704415, 0.0736245..."
1,爱好,àihào,1,"love, like, be fond of, be keen on","[0.44047022, 0.29555959, 0.5669609, -0.0190896..."
2,爸爸,bàba,1,"old man, father, papa, pappa, daddy, pa, beget...","[0.052221924, 0.18916291, 0.25977138, -0.58558..."
3,白,bái,1,"white, clear, pure, plain, wrongly written/mis...","[-0.17480367, 0.23118936, -0.27152467, 0.10098..."
4,八,bā,1,det.: eight,"[0.25673094, 0.25089708, 0.53159064, -0.352917..."


In [5]:
tokens_pd.head()

,0
0,图像
1,来源
2,Getty
3,
4,Images


In [10]:
tokens_pd.iloc[0,0]

'图像'